In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os,sys,inspect
import gc
from tqdm import tqdm
import random
import heapq

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from load import *
def eval_NDCG(true, pred):
    top_k = pred

    for i, item in enumerate(top_k, 1):
        if item == true:
            return 1 / np.log2(i+1)
    return 0

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras as K
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Embedding, Flatten, Multiply
from tensorflow.keras.models import Model, Sequential, load_model

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def leakyrelu(x, factor=0.2):
    return tf.maximum(x, factor*x)

In [3]:
df = load_data('../data/ml-100k/u.data', threshold=3)
df = df[df['rating']==1].reset_index(drop=True)
tdf = pd.pivot_table(df, index='userId', values='rating', columns='movieId').fillna(0)

# 10개 이상 평가한 유저만 포함 => 0이 나오는 문제가 발생하여
cnt = tdf.sum(1)
df = df[df['userId'].isin(np.where(cnt >= 10)[0])].reset_index(drop=True)
tdf = pd.pivot_table(df, index='userId', values='rating', columns='movieId').fillna(0)
tdf.iloc[:,:] = 0

In [4]:
test_idx = []
for i in tdf.index:
    test_idx += list(np.random.choice(df[df['userId']==i].index, 1))
    
train = df.iloc[list(set(df.index)-set(test_idx)),:]
test = df.iloc[test_idx, :]

In [5]:
for uid, iid in zip(train['userId'].values, train['movieId'].values):
    tdf.loc[uid, iid] = 1
train =  tdf.copy()

In [6]:
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.random.normal(shape=(batch, dim), stddev=0.01)
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [7]:
class MultVAE(tf.keras.models.Model):
    def __init__(self, input_dim, latent_dim, lamda=1e-4):
        super().__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.anneal = 0.
        
        self.model = self.build()

    def compile(self, optimizer, loss_fn=None):
        super().compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn

        
    def build(self):
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        
        inputs = self.encoder.input
        
        mu, log_var = self.encoder(inputs)
        h = sampling([mu, log_var])
        
        outputs = self.decoder(h)
    
        return Model(inputs, outputs)
    
    def build_encoder(self):
        inputs = Input(shape = (self.input_dim, ))
        h = Dropout(0.2)(inputs)
        
        mu = Dense(self.latent_dim)(h)
        log_var = Dense(self.latent_dim)(h)
        
        return Model(inputs, [mu, log_var])
    
    def build_decoder(self):
        inputs = Input(shape = (self.latent_dim, ))
        
        outputs = Dense(self.input_dim, activation='sigmoid')(inputs)

        return Model(inputs, outputs)
    
    def train_step(self, data):
        x = data
        with tf.GradientTape() as tape:
            mu, log_var = self.encoder(x)
            pred = self.model(x)
            
            kl_loss = tf.reduce_mean(tf.reduce_sum(0.5*(log_var + tf.exp(log_var) + tf.pow(mu, 2)-1), 1, keepdims=True))
            ce_loss = -tf.reduce_mean(tf.reduce_sum(tf.nn.log_softmax(pred) * x, -1))
            
            loss = ce_loss + kl_loss*self.anneal
            
        grads = tape.gradient(loss, self.model.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
        
        return {'loss': loss}
    
    def predict(self, data):
        mu, log_var = self.encoder(data)
        return self.decoder(mu)
    

In [8]:
loader = tf.data.Dataset.from_tensor_slices(train.values.astype(np.float32))
loader = loader.batch(8, drop_remainder=True).shuffle(len(train))

In [9]:
model = MultVAE(train.shape[1], 200)
model.compile(optimizer=tf.optimizers.Adam())

In [10]:
# anneal 값에 따라서 성능변화가 심함.. 고찰 필요
class AnnealCallback(callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.anneal_cap = 0.3
        
    def on_train_batch_end(self, batch, logs=None):
        self.model.anneal =  min(self.anneal_cap, self.model.anneal+1e-4)

In [11]:
model.fit(loader,
         epochs=25,
         callbacks=[AnnealCallback()])

Epoch 1/25
112/112 [==============================] - 0s 2ms/step - loss: 413.5160
Epoch 2/25
112/112 [==============================] - 0s 2ms/step - loss: 408.8344
Epoch 3/25
112/112 [==============================] - 0s 2ms/step - loss: 403.9771
Epoch 4/25
112/112 [==============================] - 0s 2ms/step - loss: 399.3782
Epoch 5/25
112/112 [==============================] - 0s 2ms/step - loss: 400.7264
Epoch 6/25
112/112 [==============================] - 0s 2ms/step - loss: 399.8045
Epoch 7/25
112/112 [==============================] - 0s 2ms/step - loss: 398.2963
Epoch 8/25
112/112 [==============================] - 0s 2ms/step - loss: 399.7597
Epoch 9/25
112/112 [==============================] - 0s 2ms/step - loss: 397.7665
Epoch 10/25
112/112 [==============================] - 0s 2ms/step - loss: 393.5316
Epoch 11/25
112/112 [==============================] - 0s 2ms/step - loss: 393.7732
Epoch 12/25
112/112 [==============================] - 0s 2ms/step - loss: 393.8577
E

In [12]:
top_k = 10

scores = []
for idx, i in tqdm(enumerate(train.index)):
    item_to_pred = {item: pred for item, pred in zip(train.columns, model.model.predict(train.values)[idx])}
    test_ = test[(test['userId']==i) & (test['rating']==1)]['movieId'].values
    items = list(np.random.choice(list(filter(lambda x: x not in np.argwhere(train.values[idx]).flatten(), item_to_pred.keys())), 100)) + list(test_)
    top_k_items = heapq.nlargest(top_k, items, key=item_to_pred.get)
    
    score = eval_NDCG(test_, top_k_items)
    scores.append(score)
    
np.mean(scores)

896it [01:53,  7.90it/s]


0.3934388168754929

In [13]:
top_k = 10

scores = []
for idx, i in tqdm(enumerate(train.index)):
    item_to_pred = {item: pred for item, pred in zip(train.columns, model.predict(train.values)[idx])}
    test_ = test[(test['userId']==i) & (test['rating']==1)]['movieId'].values
    items = list(np.random.choice(list(filter(lambda x: x not in np.argwhere(train.values[idx]).flatten(), item_to_pred.keys())), 100)) + list(test_)
    top_k_items = heapq.nlargest(top_k, items, key=item_to_pred.get)
    
    score = eval_NDCG(test_, top_k_items)
    scores.append(score)
    
np.mean(scores)

896it [04:22,  3.41it/s]


0.4001024033054037